In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="2oeGI3QWbiYo6HNrErIp")
project = rf.workspace("aaaa-g4pgj").project("my-first-project-pv3aw")
version = project.version(11)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-11 in yolov8:: 100%|██████████| 50878/50878 [00:27<00:00, 1871.00it/s]


In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# --- Preprocessing function you already tested ---
def remove_glare_reflection(img_bgr):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Adaptive glare mask
    v_blur = cv2.GaussianBlur(v, (11, 11), 0)
    thresh_val = np.percentile(v_blur, 98)
    glare_mask = cv2.inRange(v_blur, thresh_val - 5, 255)
    glare_mask = cv2.dilate(glare_mask, np.ones((5,5), np.uint8), iterations=2)
    glare_mask = cv2.GaussianBlur(glare_mask, (9, 9), 0)

    smooth = cv2.edgePreservingFilter(img_bgr, flags=1, sigma_s=80, sigma_r=0.3)

    mask_norm = glare_mask.astype(np.float32) / 255.0
    mask_norm = np.repeat(mask_norm[..., None], 3, axis=2)
    blend = cv2.convertScaleAbs(img_bgr * (1 - mask_norm) + smooth * mask_norm)

    inpaint_mask = cv2.inRange(v, 240, 255)
    inpainted = cv2.inpaint(blend, inpaint_mask, 3, cv2.INPAINT_TELEA)

    lab = cv2.cvtColor(inpainted, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.8, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    result = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    return result


def preprocess_split(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    for f in tqdm(files, desc=f"Processing {os.path.basename(input_dir)}"):
        img = cv2.imread(os.path.join(input_dir, f))
        if img is None:
            continue
        processed = remove_glare_reflection(img)
        cv2.imwrite(os.path.join(output_dir, f), processed)


# --- Apply to all YOLO splits ---
dataset_path = dataset.location  # from Roboflow

splits = ["train", "valid", "test"]
for split in splits:
    in_dir = os.path.join(dataset_path, split, "images")
    out_dir = os.path.join(dataset_path, f"{split}_glare_fixed")
    preprocess_split(in_dir, out_dir)

print("✅ All glare-reduced images generated.")


Processing images: 100%|██████████| 1608/1608 [15:08<00:00,  1.77it/s]

✅ All glare-reduced images generated.


In [3]:
import yaml, os

data_yaml = {
    'train': os.path.join(dataset_path, 'train_glare_fixed'),
    'val': os.path.join(dataset_path, 'valid_glare_fixed'),
    'test': os.path.join(dataset_path, 'test_glare_fixed'),
    'names': ['trash', 'bottle', 'plastic_bag', 'can']  # adjust based on your dataset
}

yaml_path = os.path.join(dataset_path, 'data_glare_fixed.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ Created YAML file at:", yaml_path)


✅ Created YAML file at: c:\Users\User\Documents\Rabab\repo\final-project\scenario3\My-First-Project-11\data_glare_fixed.yaml


In [4]:
def exposure_fuse(bgr):
    # simulate bracketed exposures
    def gamma(img, g):
        inv = 1.0 / g
        table = np.array([(i/255.0)**inv*255 for i in range(256)]).astype("uint8")
        return cv2.LUT(img, table)
    imgs = [gamma(bgr, 0.7), bgr, gamma(bgr, 1.5)]
    merge = cv2.createMergeMertens().process([i.astype(np.float32)/255.0 for i in imgs])
    return (np.clip(merge,0,1)*255).astype(np.uint8)